In [112]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import ttest_ind, f_oneway
import os
import yfinance as yf

In [115]:
stock = yf.Ticker('AAPL')

In [116]:
stock = stock.history(start='2019-01-01', end='2024-08-22')

In [117]:
stock.shape

(1419, 7)

In [118]:
len = stock.shape[0]
len

1419

In [119]:
stock['52 Week High'] = stock['High'].rolling(window=365).max()
stock['52 Week Low'] = stock['Low'].rolling(window=365).min()

In [120]:
stock

,Open,High,Low,Close,Volume,Dividends,Stock Splits,52 Week High,52 Week Low
Date,,,,,,,,,
2019-01-02 00:00:00-05:00,37.025776,37.972398,36.868005,37.750084,148158800,0.0,0.0,NaN,NaN
2019-01-03 00:00:00-05:00,34.417789,34.833729,33.944479,33.989899,365248800,0.0,0.0,NaN,NaN
2019-01-04 00:00:00-05:00,34.549267,35.510232,34.374765,35.440907,234428400,0.0,0.0,NaN,NaN
2019-01-07 00:00:00-05:00,35.546092,35.577170,34.876764,35.362026,219111200,0.0,0.0,NaN,NaN
2019-01-08 00:00:00-05:00,35.751664,36.291910,35.503058,36.036129,164101200,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-08-15 00:00:00-04:00,224.600006,225.350006,222.759995,224.720001,46414000,0.0,0.0,236.955728,146.457905
2024-08-16 00:00:00-04:00,223.919998,226.830002,223.649994,226.050003,44340200,0.0,0.0,236.955728,146.457905
2024-08-19 00:00:00-04:00,225.720001,225.990005,223.039993,225.889999,40687800,0.0,0.0,236.955728,146.457905


In [122]:
stock['Sentiment'] = (stock['Close'] >= stock['Open']).astype('int')

In [123]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1419 entries, 2019-01-02 00:00:00-05:00 to 2024-08-21 00:00:00-04:00
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1419 non-null   float64
 1   High          1419 non-null   float64
 2   Low           1419 non-null   float64
 3   Close         1419 non-null   float64
 4   Volume        1419 non-null   int64  
 5   Dividends     1419 non-null   float64
 6   Stock Splits  1419 non-null   float64
 7   52 Week High  1055 non-null   float64
 8   52 Week Low   1055 non-null   float64
 9   Sentiment     1419 non-null   int64  
dtypes: float64(8), int64(2)
memory usage: 121.9 KB


In [124]:
stock = stock[366:]

In [125]:
from sklearn.preprocessing import StandardScaler

In [126]:
scaler  = StandardScaler()

In [129]:
from sklearn.preprocessing import PowerTransformer

In [130]:
pt = PowerTransformer()

In [131]:
stock['Open'] = scaler.fit_transform(stock[['Open']])

<ipython-input-131-6549ba3c2dec>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [132]:
stock['High'] = scaler.fit_transform(stock[['High']])
stock['Low'] = scaler.fit_transform(stock[['Low']])
stock['Close'] = scaler.fit_transform(stock[['Close']])
stock['Volume'] = np.log(stock[['Volume']])
stock['52 Week High']=pt.fit_transform(stock[['52 Week High']])
stock['52 Week Low']=pt.fit_transform(stock[['52 Week Low']])

<ipython-input-132-1c90fea1026f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-132-1c90fea1026f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-132-1c90fea1026f>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-132-1c9

In [168]:
df = stock

In [150]:
df.shape

(1053, 10)

In [135]:
highest = df.iloc[1052,7]

In [136]:
lowest = df.iloc[1052,8]

In [137]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,52 Week High,52 Week Low,Sentiment
Date,,,,,,,,,,
2020-06-16 00:00:00-04:00,-2.349290,-2.382537,-2.354708,-2.343501,2.940434,0.0,0.0,-2.418908,-1.601912,1
2020-06-17 00:00:00-04:00,-2.318410,-2.364210,-2.301307,-2.347593,2.920754,0.0,0.0,-2.415867,-1.593853,0
2020-06-18 00:00:00-04:00,-2.349708,-2.380455,-2.316983,-2.346423,2.911718,0.0,0.0,-2.415867,-1.583692,1
2020-06-19 00:00:00-04:00,-2.322677,-2.354547,-2.351103,-2.363210,2.964926,0.0,0.0,-2.410257,-1.580957,0
2020-06-22 00:00:00-04:00,-2.350293,-2.330388,-2.300803,-2.286793,2.929810,0.0,0.0,-2.396165,-1.580957,1


In [138]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

In [169]:
from sklearn.model_selection import train_test_split
# Slicing the DataFrame into training and testing sets
train_data = df[:820]
test_data = df[820:]


In [170]:
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(data.shape[0] - time_step - 1):
        if isinstance(data, pd.DataFrame):
            X.append(data.iloc[i:(i + time_step), 0].values)
            y.append(data.iloc[i + time_step, 0])
        else:
            X.append(data[i:(i + time_step), 0])
            y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Create training and testing datasets
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)


In [171]:
fig = px.line(df, x=df.index, y='Close', title='Closing Price Over Time')
fig.update_layout(title_text='Closing Price Over Time', title_x=0.5, template='plotly_dark')
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [172]:
fig = make_subplots(rows=3, cols=2, subplot_titles=('Close', 'Volume', 'Open', 'High', 'Low', 'Adj Close'))
fig.add_trace(go.Histogram(x=df['Close'], name='Close'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Volume'], name='Volume'), row=1, col=2)
fig.add_trace(go.Histogram(x=df['Open'], name='Open'), row=2, col=1)
fig.add_trace(go.Histogram(x=df['High'], name='High'), row=2, col=2)
fig.add_trace(go.Histogram(x=df['Low'], name='Low'), row=3, col=1)
fig.update_layout(title_text='Distribution of Features', title_x=0.5, template='plotly_dark')
fig.show()

In [161]:
Y = df['Close']
X = df.drop(['Close'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [37]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [173]:
history = model.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
759/759 [==============================] - 19s 25ms/step - loss: 0.0247
Epoch 2/10
759/759 [==============================] - 20s 27ms/step - loss: 0.0236
Epoch 3/10
759/759 [==============================] - 19s 24ms/step - loss: 0.0228
Epoch 4/10
759/759 [==============================] - 18s 24ms/step - loss: 0.0243
Epoch 5/10
759/759 [==============================] - 19s 25ms/step - loss: 0.0229
Epoch 6/10
759/759 [==============================] - 18s 24ms/step - loss: 0.0192
Epoch 7/10
759/759 [==============================] - 19s 25ms/step - loss: 0.0213
Epoch 8/10
759/759 [==============================] - 18s 24ms/step - loss: 0.0222
Epoch 9/10
759/759 [==============================] - 20s 26ms/step - loss: 0.0204
Epoch 10/10
759/759 [==============================] - 21s 28ms/step - loss: 0.0205


In [174]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

6/6 [==============================] - 0s 19ms/step


In [175]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform([y_train])
y_test = scaler.inverse_transform([y_test])

In [176]:
train_rmse = np.sqrt(mean_squared_error(y_train[0], train_predict[:, 0]))
train_mae = mean_absolute_error(y_train[0], train_predict[:, 0])
test_rmse = np.sqrt(mean_squared_error(y_test[0], test_predict[:, 0]))
test_mae = mean_absolute_error(y_test[0], test_predict[:, 0])

print(f'Train RMSE: {train_rmse}, Train MAE: {train_mae}')
print(f'Test RMSE: {test_rmse}, Test MAE: {test_mae}')

Train RMSE: 0.4986788610386477, Train MAE: 0.38343022128760845
Test RMSE: 1.3309040028590984, Test MAE: 0.9469166247535034


In [177]:
train_data_len = len(train_data)

train_plot = np.empty_like(scaled_data)
train_plot[:, :] = np.nan
train_plot[time_step:len(train_predict)+time_step, :] = train_predict

test_plot = np.empty_like(scaled_data)
test_plot[:, :] = np.nan
test_plot[len(train_predict)+(time_step*2)+1:len(scaled_data)-1, :] = test_predict

TypeError: 'int' object is not callable

In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Actual Price', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index, y=train_plot[:, 0], mode='lines', name='Train Predict', line=dict(color='green')))
fig.add_trace(go.Scatter(x=df.index, y=test_plot[:, 0], mode='lines', name='Test Predict', line=dict(color='red')))
fig.update_layout(title='Stock Price Prediction',
                   xaxis_title='Date',
                   yaxis_title='Stock Price',
                   template='plotly_dark')
fig.show()

In [49]:
def predict_future(model, data, scaler, time_step=60, future_steps=30):
    # Prepare the input data for future prediction
    last_data = df[-time_step:]
    last_data = last_data.reshape(1, time_step, 1)

    # Predict future prices
    pred = model.predict(last_data)
    future_predictions.append(next_pred[0, 0])
    print(pred)

In [51]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 70.4 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-cp310-cp310-linux_x86_64.whl size=728064 sha256=da6001cdaabecacf31eceb29e183fbcdb66b752bc8767e5bcea78a7b8e0dcb33
  Stored in directory: /root/.cache/pip/wheels/4b/b9/b0/83d6e258e8f963f5ff111a2cd8c483ca59372a86e6a2535212
Successfully built peewee


In [52]:
import yfinance as yf

In [178]:
ticker_symbol = 'AAPL'  # Example: Apple Inc.

# Fetch the stock data
stock = yf.Ticker(ticker_symbol)

# Get the current market price
Close = stock.history(period='1d')['Close'][0]
Open = stock.history(period='1d')['Open'][0]
High = stock.history(period='1d')['High'][0]
Low  =  stock.history(period='1d')['Low'][0]
Volume = stock.history(period='1d')['Volume'][0]
close = scaler.fit_transform(c)

<ipython-input-178-89700b0a02d6>:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-178-89700b0a02d6>:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-178-89700b0a02d6>:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-178-89700b0a02d6>:10: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consiste

In [179]:
X = np.array([[Open, High, Low, Close, Volume,0,0,highest,lowest,1]])

In [66]:
X[0][0]

225.66000366210938

In [84]:
X_train.shape

(781, 60)

In [80]:
X_train[780]

array([1.30372481, 1.30099812, 1.31258905, 1.39271139, 1.34668437,
       1.39407478, 1.40021198, 1.35929857, 1.24099059, 1.03472015,
       0.91914016, 0.83594884, 0.8761807 , 0.82878905, 0.76330997,
       0.78550065, 0.81656881, 0.75682265, 0.75716414, 0.59192459,
       0.68649305, 0.75443215, 0.80427794, 0.87767951, 0.76535758,
       0.85787807, 0.84456291, 1.02345961, 1.12246648, 1.17879885,
       1.13748867, 1.14158565, 0.69024794, 0.79847364, 0.85719567,
       0.83739416, 0.73565498, 0.64996282, 0.73463129, 0.73463108,
       0.77013747, 0.82954133, 0.66873971, 0.67283689, 0.65679104,
       0.67795816, 0.60284896, 0.4908681 , 0.58236459, 0.55505183,
       0.59055799, 0.5506139 , 0.64279312, 0.64313476, 0.74589766,
       0.78993908, 0.79335252, 0.85719538, 0.90328493, 0.74384824])

In [180]:
model.predict(X)

1/1 [==============================] - 0s 27ms/step


array([[1.1559821]], dtype=float32)

In [181]:
VAL = scaler.inverse_transform(model.predict(X))
VAL

1/1 [==============================] - 0s 27ms/step


array([[3.5456634]], dtype=float32)

In [88]:
ticker_symbol = 'AAPL'  # Example: Apple Inc.

# Fetch the stock data
stock1 = yf.Ticker(ticker_symbol)

In [89]:
stock = stock1.history(interval='1m', period='1d')

In [90]:
stock

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-08-23 09:30:00-04:00,225.658905,226.100006,225.520004,225.869995,1048607,0.0,0.0
2024-08-23 09:31:00-04:00,225.910004,226.039993,225.580002,225.899994,138669,0.0,0.0
2024-08-23 09:32:00-04:00,225.904999,226.009995,225.649994,226.000000,121312,0.0,0.0
2024-08-23 09:33:00-04:00,225.994995,226.070007,225.630005,225.669998,156151,0.0,0.0
2024-08-23 09:34:00-04:00,225.674896,226.089096,225.674896,226.020004,100158,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-23 15:55:00-04:00,227.050003,227.100006,226.729996,226.789993,205064,0.0,0.0
2024-08-23 15:56:00-04:00,226.800003,226.914993,226.699997,226.914001,129052,0.0,0.0
2024-08-23 15:57:00-04:00,226.919998,226.970001,226.800003,226.830002,164026,0.0,0.0


In [ ]:
stock['52 week high'] = highest
stock['52 week low'] = lowest

In [93]:
from sklearn.preprocessing import StandardScaler
import numpy as np

In [94]:
scaler  = StandardScaler()

In [95]:
stock['Open'] = scaler.fit_transform(stock[['Open']])

In [97]:
stock['High'] = scaler.fit_transform(stock[['High']])
stock['Low'] = scaler.fit_transform(stock[['Low']])
stock['Close'] = scaler.fit_transform(stock[['Close']])
stock['Volume'] = np.log(stock[['Volume']])
stock['52 Week High']=pt.fit_transform(stock[['52 Week High']])
stock['52 Week Low']=pt.fit_transform(stock[['52 Week Low']])

NameError: name 'pt' is not defined